In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rexpi
import scipy.sparse

# Matrix exponential, diagonal case similar to scalar setting

compute $\exp(\mathrm{i} \omega H )u$ for a matrix $H\in\mathbb{C}^{k\times k}$ and vector $u\in\mathbb{C}^k$.

In [2]:
n = 40
tol = 1e-6
nk=10

step_factor = 0.02
nodes_pos=None
tol=1e-6

tolequi = 1e-3
w = rexpi.buerrest_getw(n,tol)
#w=730.0
pchebnodes = rexpi.PositiveChebyshevNodes(n)
wmax = (n+1)*np.pi
pws = np.linspace(1,n/2,int(n/2))
px=np.concatenate((pws,n/2+0.5*pws))
trys= n/px[::-1]

#snodes = pchebnodes + (-1+(n+2+np.arange(n))/(n+1)-pchebnodes)*(w/wmax)**trys
#snodes=gn
print("for n=%d and tol=%g, our error estimate suggests w=%f"%(n,tol,w))
print("run brib algorithm with tol.rel.dev = %.2e, stp.fac. = %.2e" %(tolequi,step_factor))
rx, brnodes, allerr = rexpi.brib(w = w, n = n, npi=50, nodes_pos = nodes_pos,
                                   maxiter = 1200, tolequi = tolequi, step_factor=step_factor,syminterp=True)
errlast = allerr[-1][0]
rdev = allerr[-1][1]
rdev_old = allerr[-1][2]
usediter = len(allerr)
intnodes, equnodes = brnodes[1],brnodes[0]
nodes_pos = intnodes[intnodes>=0]
print("   .. done, used %d iterations, rel. deviation = %.2e" % (usediter,rdev))
print("approximation error = %.8e"%(errlast))

poles = rx.getpoles(sym=True)
a0, aj, sj = rx.getpartialfractioncoef(sym=True)

# define diagonal matrix with spectrum between -1 and 1
k=100
u = np.ones(k)
# in this example the matrix H corresponds to the diagonal matrix
eigsH = np.linspace(-1,1,k)
H = scipy.sparse.diags([eigsH], [0])

yscalar = rx(1j*eigsH)
yref = np.exp(1j*w*eigsH)

print("evaluate scalar r: max|r(ix)-exp(iwx)| = %.2e" % np.max(np.abs(yref-yscalar)))

mviH = lambda x : 1j*eigsH*x
mvSaIiH = lambda s,x : (1j*eigsH-s)**(-1)*x
#mvSaIiH = lambda s,x : np.linalg.solve(1j*H - s*np.eye(k),x)
#mvSaIiH = lambda s,x : (np.linalg.inv(1j*H.toarray() - s*np.eye(k))).dot(x)

c0=(-1)**(n)
ypoles = rexpi.evalr_product(mviH, mvSaIiH, u, poles, c0)
print("use poles: max|r(ix)-exp(iwx)| = %.2e" % np.max(np.abs(yref-ypoles)))

yparfrac = rexpi.evalr_partialfraction(mvSaIiH, u, a0, aj, sj)
errpf = np.max(np.abs(yref-yparfrac))
print("max |aj| = %.2e, min Re(sj) = %.2e" % (max(np.abs(aj)),min(sj.real)))
print("use partial fraction: max|r(ix)-exp(iwx)| = %.2e" % errpf)
print("(computing r(iH)u via partial fraction is only accurate for r of small degree)")

for n=40 and tol=1e-06, our error estimate suggests w=99.651100
run brib algorithm with tol.rel.dev = 1.00e-03, stp.fac. = 2.00e-02
   .. done, used 142 iterations, rel. deviation = 9.99e-04
approximation error = 2.08645340e-07
evaluate scalar r: max|r(ix)-exp(iwx)| = 2.08e-07
use poles: max|r(ix)-exp(iwx)| = 2.08e-07
max |aj| = 2.04e+04, min Re(sj) = 5.77e-02
use partial fraction: max|r(ix)-exp(iwx)| = 2.08e-07
(computing r(iH)u via partial fraction is only accurate for r of small degree)


In [3]:
# define diagonal matrix with spectrum between -1 and 1
k=9800
u = np.ones(k)
# in this example the matrix H corresponds to the diagonal matrix
eigsH = np.linspace(-1,1,k)
yref = np.exp(1j*w*eigsH)

eigsH = eigsH+np.random.rand(k)*1e-12
H = scipy.sparse.diags([eigsH], [0])

yscalar = rx(1j*eigsH)

print("evaluate scalar r: max|r(ix)-exp(iwx)| = %.2e" % np.max(np.abs(yref-yscalar)))

mviH = lambda x : 1j*eigsH*x
mvSaIiH = lambda s,x : (1j*eigsH-s)**(-1)*x
#mvSaIiH = lambda s,x : np.linalg.solve(1j*H - s*np.eye(k),x)
#mvSaIiH = lambda s,x : (np.linalg.inv(1j*H.toarray() - s*np.eye(k))).dot(x)

c0=(-1)**(n)
ypoles = rexpi.evalr_product(mviH, mvSaIiH, u, poles, c0)
print("use poles: max|r(ix)-exp(iwx)| = %.2e" % np.max(np.abs(yref-ypoles)))

yparfrac = rexpi.evalr_partialfraction(mvSaIiH, u, a0, aj, sj)
print("max |aj| = %.2e, min Re(sj) = %.2e" % (max(np.abs(aj)),min(sj.real)))
print("use partial fraction: max|r(ix)-exp(iwx)| = %.2e" % np.max(np.abs(yref-yparfrac)))
print("(computing r(iH)u via partial fraction is only accurate for r of small degree)")

evaluate scalar r: max|r(ix)-exp(iwx)| = 2.09e-07
use poles: max|r(ix)-exp(iwx)| = 2.09e-07
max |aj| = 2.04e+04, min Re(sj) = 5.77e-02
use partial fraction: max|r(ix)-exp(iwx)| = 2.09e-07
(computing r(iH)u via partial fraction is only accurate for r of small degree)


### compare with Pade

In [4]:
npade = n
rpade = rexpi.pade(npade)

# test scalar approximation error of pade
ypadescalar = rpade(1j*w*eigsH)

# evaluate r(iwH), argument has to be re-scaled by w
poles2 = rpade.getpoles()
mviHscaled = lambda x : 1j*w*eigsH*x
mvSaIiHscaled = lambda s,x : (1j*w*eigsH-s)**(-1)*x
c02 = (-1)**(npade)
ypadepoles = rexpi.evalr_product(mviHscaled, mvSaIiHscaled, u, poles2, c02)
ap0, apj, spj = rpade.getpartialfractioncoef()

ypadeparfrac = rexpi.evalr_partialfraction(mvSaIiHscaled, u, ap0, apj, spj)

print("approximate exponential with Pade, same problem as above")
print("scalar error: max|r(iwx)-exp(iwx)| = %.2e" % np.max(np.abs(yref-ypadescalar)))
print("use poles: max|r(iwx)-exp(iwx)| = %.2e" % np.max(np.abs(yref-ypadepoles)))
print("use partial fraction: max|r(iwx)-exp(iwx)| = %.2e" % np.max(np.abs(yref-ypadeparfrac)))

approximate exponential with Pade, same problem as above
scalar error: max|r(iwx)-exp(iwx)| = 2.00e+00
use poles: max|r(iwx)-exp(iwx)| = 2.00e+00
use partial fraction: max|r(iwx)-exp(iwx)| = 1.04e+15


### compare with polynomial Chebyshev

In [5]:
ncheb = 30
# evaluate Chebyshev approximation using Clenshaw Algorithm
ypcheb = rexpi.eval_polynomial_chebyshev(eigsH, w, ncheb)
# evaluate with mv using Clenshaw Algorithm
mvH = lambda x : eigsH*x
ypcheb2 = rexpi.chebyshev(mvH,w,u,ncheb)

print("approximate exponential with Pade, same problem as above")
print("scalar error: max|r(iwx)-exp(iwx)| = %.2e" % np.max(np.abs(yref-ypcheb)))
print("by mv: max|r(iwx)-exp(iwx)| = %.2e" % np.max(np.abs(yref-ypcheb2)))

approximate exponential with Pade, same problem as above
scalar error: max|r(iwx)-exp(iwx)| = 1.22e+00
by mv: max|r(iwx)-exp(iwx)| = 1.22e+00


## Tridiagonal matrix, errors in norm

In [6]:
# with direct solver, small dimension
nrm = lambda x : np.linalg.norm(x)

print("use brib for n=%d, tol=%g, and w=%f"%(n,tol,w))

# define diagonal matrix with spectrum between -1 and 1
k=50
print("random starting vector u with ||u||=1, dimension k=%d" %k)
u = np.random.rand(k)
u = u/nrm(u)

e1 = np.ones(k-1)
e = np.ones(k)
o = np.zeros(k)

# H is shifted Laplace operator with eigenvalues between -1 and 1
H = scipy.sparse.diags([0.5*e1,o,0.5*e1], [-1,0,1])
yref = scipy.sparse.linalg.expm_multiply(1j*w*H,u)

mviH = lambda x : 1j*H.dot(x)
# apply numpy solve to apply x -> (H-s*I)**-1
#mvSaIiH = lambda s,x : np.linalg.solve(1j*H - s*np.eye(k),x)
# apply the matrix inverse of H-s*I
mvSaIiH = lambda s,x : (np.linalg.inv(1j*H.toarray() - s*np.eye(k))).dot(x)

poles = rx.getpoles(sym=True)
c0=(-1)**(n)
ypoles = rexpi.evalr_product(mviH, mvSaIiH, u, poles, c0)
print("use poles: ||r(ix)-exp(iwx)|| = %.2e" % nrm(yref-ypoles))

a0, aj, sj = rx.getpartialfractioncoef(sym=True)
yparfrac = rexpi.evalr_partialfraction(mvSaIiH, u, a0, aj, sj)
print("use partial fraction: ||r(ix)-exp(iwx)|| = %.2e" % nrm(yref-yparfrac))
print("(error in l2 norm is bounded by scalar error since the spectrum of H is in [-1,1] and ||u||=1)")
print("(computing r(iH)u via partial fraction is only accurate for r of small degree)")

use brib for n=40, tol=1e-06, and w=99.651100
random starting vector u with ||u||=1, dimension k=50
use poles: ||r(ix)-exp(iwx)|| = 1.89e-07
use partial fraction: ||r(ix)-exp(iwx)|| = 1.89e-07
(error in l2 norm is bounded by scalar error since the spectrum of H is in [-1,1] and ||u||=1)
(computing r(iH)u via partial fraction is only accurate for r of small degree)


### use sparse solver

In [7]:
# sparse solver, large dimension
nrm = lambda x : np.linalg.norm(x)

print("use brib for n=%d, tol=%g, and w=%f"%(n,tol,w))

# define diagonal matrix with spectrum between -1 and 1
k=10000
print("random starting vector u with ||u||=1, dimension k=%d"%k)
u = np.random.rand(k)
u = u/nrm(u)

e1 = np.ones(k-1)
e = np.ones(k)
o = np.zeros(k)

# H is shifted Laplace operator with eigenvalues between -1 and 1
H = scipy.sparse.diags([0.5*e1,o,0.5*e1], [-1,0,1])
yref = scipy.sparse.linalg.expm_multiply(1j*w*H,u)

mviH = lambda x : 1j*H.dot(x)
# use a banded sparse solver to compute the inverse of H-s*I
mvSaIiH =lambda s,x : scipy.sparse.linalg.spsolve(scipy.sparse.diags([0.5j*e1,-s*e,0.5j*e1], [-1,0,1]).tocsc(), x)

poles = rx.getpoles(sym=True)
c0=(-1)**(n)
ypoles = rexpi.evalr_product(mviH, mvSaIiH, u, poles, c0)
print("use poles: max|r(ix)-exp(iwx)| = %.2e" % nrm(yref-ypoles))

a0, aj, sj = rx.getpartialfractioncoef(sym=True)
yparfrac = rexpi.evalr_partialfraction(mvSaIiH, u, a0, aj, sj)
print("max|aj| = %.2e" % max(np.abs(aj)))
print("use partial fraction: ||r(ix)-exp(iwx)|| = %.2e" % nrm(yref-yparfrac))
print("(error in l2 norm is bounded by scalar error since the spectrum of H is in [-1,1] and ||u||=1)")
print("(computing r(iH)u via partial fraction is only accurate for r of small degree)")

use brib for n=40, tol=1e-06, and w=99.651100
random starting vector u with ||u||=1, dimension k=10000
use poles: max|r(ix)-exp(iwx)| = 1.94e-07
max|aj| = 2.04e+04
use partial fraction: ||r(ix)-exp(iwx)|| = 1.94e-07
(error in l2 norm is bounded by scalar error since the spectrum of H is in [-1,1] and ||u||=1)
(computing r(iH)u via partial fraction is only accurate for r of small degree)
